# Simple Chatbot Implementation

In [43]:
import pandas as pd

# Load the financial dataset
df = pd.read_csv("transformed_financial_data.csv")

# Sort by year just in case
df = df.sort_values(by=['Company', 'Fiscal Year'])


def simple_chatbot(user_query):
    # Normalize for case-insensitive handling
    query = user_query.lower()
    company = extract_company(query)
    year = extract_year(query)

    if "total revenue" in query:
        if company:
            if year:
                revenue = df[(df['Company'] == company) & (df['Fiscal Year'] == year)]['Total Revenue (USD M)'].values[0]
                return f"{company}'s total revenue in {year} was ${revenue:,.0f} million."
            else:
                latest_year = df[df['Company'] == company]['Fiscal Year'].max()
                revenue = df[(df['Company'] == company) & (df['Fiscal Year'] == latest_year)]['Total Revenue (USD M)'].values[0]
                return f"{company}'s total revenue in {latest_year} (latest year) was ${revenue:,.0f} million."
        else:
            return "Please specify the company name."

    elif "net income" in query and "change" in query:
        if company:
            company_df = df[df['Company'] == company].sort_values(by='Fiscal Year')
            ni_change = company_df['Net Income Growth (%)'].dropna().iloc[-1] * 100
            prev_year = company_df['Fiscal Year'].iloc[-2]
            latest_year = company_df['Fiscal Year'].iloc[-1]
            return f"{company}'s net income changed by {ni_change:.2f}% from {prev_year} to {latest_year}."
        else:
            return "Please specify the company name."
    
    elif "asset to liability ratio" in query or "asset liability ratio" in query:
        if company:
            if year:
                row = df[(df['Company'] == company) & (df['Fiscal Year'] == int(year))]
                ratio = row['Asset_Liability_Ratio'].values[0]
                return f"{company}'s asset to liability ratio was {ratio:.2f} in {year}."
            else:
                company_data = df[df['Company'] == company]
                latest_row = company_data.sort_values('Fiscal Year', ascending=False).iloc[0]
                ratio = latest_row['Asset_Liability_Ratio']
                latest_year = latest_row['Fiscal Year']
                return f"{company}'s asset to liability ratio was {ratio:.2f} in {latest_year} (latest year)."
        else:
            return "Please specify the company name."


    elif "operating cash flow" in query or "cash flow from operating activities" in query:
        if company:
            if year:
                row = df[(df['Company'] == company) & (df['Fiscal Year'] == int(year))]
                cf = row['Cash Flow from Operating Activities (USD M)'].values[0]
                return f"{company}'s operating cash flow in {year} was ${cf:,.0f} million."
            else:
                latest_year = df[df['Company'] == company]['Fiscal Year'].max()
                cf = df[(df['Company'] == company) & (df['Fiscal Year'] == latest_year)]['Cash Flow from Operating Activities (USD M)'].values[0]
                return f"{company}'s operating cash flow in {latest_year} (latest year) was ${cf:,.0f} million."
        else:
            return "Please specify the company name."
    
    elif "highest asset to liability" in query:
        latest_df = df[df['Fiscal Year'] == df['Fiscal Year'].max()]
        latest_df = latest_df.copy()
        latest_df['Ratio'] = latest_df['Total Assets (USD M)'] / latest_df['Total Liabilities (USD M)']
        top_company = latest_df.loc[latest_df['Ratio'].idxmax()]
        return f"{top_company['Company']} has the highest asset to liability ratio of {top_company['Ratio']:.2f} in {int(top_company['Fiscal Year'])}."

    else:
        return "Sorry, I can only answer questions like total revenue, net income change, operating cash flow, or asset/liability ratios."


# Helper Functions to Extract Company Name and Year
def extract_company(query):
    if "apple" in query:
        return "Apple"
    elif "microsoft" in query:
        return "Microsoft"
    elif "tesla" in query:
        return "Tesla"
    return None

def extract_year(query):
    if '2024' in query:
        return 2024
    elif '2023' in query:
        return 2023
    elif '2022' in query:
        return 2022
    return None


# CLI Interaction
if __name__ == "__main__":
    print("Welcome to the Financial Chatbot. Ask about revenue, net income, cash flow, and balance ratios.")
    print("e.g., What is the total revenue for Apple?")
    print("Type 'exit' to quit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break
        response = simple_chatbot(user_input)
        print(f"Bot: {response}")


Welcome to the Financial Chatbot. Ask about revenue, net income, cash flow, and balance ratios.
e.g., What is the total revenue for Apple?
Type 'exit' to quit.


You:  operating cash flow apple 2022


Bot: Apple's operating cash flow in 2022 was $122,151 million.


You:  exit


Bot: Goodbye!
